# Models for the preprocessed data

## Importing Packages

In [3]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import randint
from sklearn.metrics import make_scorer, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
!pip install xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
!pip install imblearn
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import get_scorer_names
from sklearn.neural_network import MLPClassifier

from tensorflow import keras

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from __future__ import print_function
import numpy as np
from sklearn import datasets




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Set random seed

In [4]:
np.random.seed(1)
tf.random.set_seed(1)

In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## Load data

In [6]:
from google.colab import drive 
drive.mount('/content/gdrive')

train_X =pd.read_csv('gdrive/My Drive/aaw_train_X.csv')
train_y =pd.read_csv('gdrive/My Drive/aaw_train_y.csv')
test_X =pd.read_csv('gdrive/My Drive/aaw_test_X.csv')
test_y =pd.read_csv('gdrive/My Drive/aaw_test_y.csv')

Mounted at /content/gdrive


## Model the data

In [7]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

## LOGISTIC REGRESSION 

### LOGISTIC REGRESSION - RANDOM SEARCH

In [8]:
score_measure = "accuracy"
kfolds = 10

param_grid = {
    'max_iter' : np.arange(1,100),
    'penalty': ['none','l1','l2','elasticnet'],
    'solver':['saga','liblinear'],
    'C': [0,0.01],
    'l1_ratio': [0.25, 0.5, 0.75],
}

logistic_reg = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = logistic_reg, param_distributions=param_grid, cv=kfolds, n_iter=50,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 10 folds for each of 50 candidates, totalling 500 fits
The best accuracy score is 0.9768476621417799
... with parameters: {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 73, 'l1_ratio': 0.5, 'C': 0.01}


### LOGISTIC REGRESSION - GRID SEARCH

In [9]:
score_measure = "accuracy"
kfolds = 10
max_iter = rand_search.best_params_['max_iter']
penalty = rand_search.best_params_['penalty']
solver = rand_search.best_params_['solver']
C = rand_search.best_params_['C']

param_grid = {  
    'max_iter' : np.arange(max_iter-5,max_iter+5),
    'penalty': [penalty],
    'solver': [solver],
    'C': [C]
}

logistic_reg = LogisticRegression()
grid_search = GridSearchCV(estimator = logistic_reg, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallLogistic = grid_search.best_estimator_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
The best accuracy score is 0.9768476621417799
... with parameters: {'C': 0.01, 'max_iter': 68, 'penalty': 'l1', 'solver': 'liblinear'}


In [10]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## SVM

### SVM CLASSIFIER - LINEAR - RBF - POLY - RANDOM SEARCH

In [11]:
score_measure = "accuracy"
kfolds = 10

param_grid = {
    'C': np.arange(1,5),   
    'gamma': ['scale','auto'],
    'kernel':['linear','rbf','poly']
}

svm = SVC()
rand_search = RandomizedSearchCV(estimator = svm, param_distributions=param_grid, cv=10, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 10 folds for each of 24 candidates, totalling 240 fits
The best accuracy score is 1.0
... with parameters: {'kernel': 'linear', 'gamma': 'scale', 'C': 1}


### SVM CLASSIFIER - LINEAR - RBF - POLY - GRID SEARCH

In [12]:
score_measure = "accuracy"
kfolds = 10

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-3,C+3),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

svm = SVC()
grid_search = GridSearchCV(estimator = svm, param_grid=param_grid, cv=10, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallSVM = grid_search.best_estimator_

Fitting 10 folds for each of 6 candidates, totalling 60 fits
The best accuracy score is 1.0
... with parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [13]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## DECISION TREE

### DECISION TREE - RANDOM SEARCH

In [14]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 50), 
    'max_depth': np.arange(1,20), 
    'criterion': ['gini', 'entropy'],
}
dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 1.0
... with parameters: {'min_samples_split': 16, 'min_samples_leaf': 29, 'min_impurity_decrease': 0.0011, 'max_leaf_nodes': 13, 'max_depth': 12, 'criterion': 'gini'}


### DECISION TREE - GRID SEARCH

In [15]:
score_measure = "accuracy"
kfolds = 5
min_samples_split = rand_search.best_params_['min_samples_split']
min_samples_leaf = rand_search.best_params_['min_samples_leaf']
min_impurity_decrease = rand_search.best_params_['min_impurity_decrease']
max_leaf_nodes = rand_search.best_params_['max_leaf_nodes']
max_depth = rand_search.best_params_['max_depth']
criterion = rand_search.best_params_['criterion']

param_grid = {
    'min_samples_split': np.arange(min_samples_split-2,min_samples_split+2),  
    'min_samples_leaf': np.arange(min_samples_leaf-2,min_samples_leaf+2),
    'min_impurity_decrease': np.arange(min_impurity_decrease-0.0001, min_impurity_decrease+0.0001, 0.00005),
    'max_leaf_nodes': np.arange(max_leaf_nodes-2,max_leaf_nodes+2), 
    'max_depth': np.arange(max_depth-2,max_depth+2), 
    'criterion': [criterion]
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 1280 candidates, totalling 6400 fits
The best accuracy score is 1.0
... with parameters: {'criterion': 'gini', 'max_depth': 10, 'max_leaf_nodes': 11, 'min_impurity_decrease': 0.001, 'min_samples_leaf': 27, 'min_samples_split': 14}


In [16]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## NEURAL NETWORKS

In [17]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(60,50,40), solver='adam', max_iter=200)
_ = ann.fit(train_X, train_y)

CPU times: user 980 ms, sys: 715 ms, total: 1.69 s
Wall time: 901 ms


In [18]:
%%time
y_pred = ann.predict(test_X)

CPU times: user 3.18 ms, sys: 1.05 ms, total: 4.22 ms
Wall time: 4.06 ms


In [19]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        86
           1       1.00      0.98      0.99       136

    accuracy                           0.99       222
   macro avg       0.98      0.99      0.99       222
weighted avg       0.99      0.99      0.99       222



### NEURAL NETWORKS - RANDOM SEARCH

In [20]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (70, 50, 40), 'alpha': 0.5, 'activation': 'tanh'}
CPU times: user 6.67 s, sys: 3.42 s, total: 10.1 s
Wall time: 3min 1s


In [21]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        86
           1       1.00      1.00      1.00       136

    accuracy                           1.00       222
   macro avg       1.00      1.00      1.00       222
weighted avg       1.00      1.00      1.00       222

CPU times: user 13.9 ms, sys: 16.2 ms, total: 30.2 ms
Wall time: 17 ms


### NEURAL NETWOKS - GRID SEARCH

In [22]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'activation': 'tanh', 'alpha': 0.7, 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.005, 'max_iter': 5000, 'solver': 'adam'}
CPU times: user 3.08 s, sys: 713 ms, total: 3.79 s
Wall time: 1min 43s


In [23]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        86
           1       1.00      0.99      1.00       136

    accuracy                           1.00       222
   macro avg       0.99      1.00      1.00       222
weighted avg       1.00      1.00      1.00       222

CPU times: user 17.8 ms, sys: 9.05 ms, total: 26.9 ms
Wall time: 26.9 ms


In [24]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Neural_Networks_Grid_search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## DEEP NEURAL NETWORKS

In [25]:
%%time

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(20))
model.add(keras.layers.Dense(15, activation='relu',kernel_initializer= tf.keras.initializers.GlorotUniform()))
model.add(keras.layers.Dense(15, activation='relu',kernel_initializer= tf.keras.initializers.GlorotUniform()))
model.add(keras.layers.Dense(15, activation='relu',kernel_initializer= tf.keras.initializers.GlorotUniform()))
model.add(keras.layers.Dense(1, activation='sigmoid')) # final layer, 10 categories


# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


CPU times: user 141 ms, sys: 48.3 ms, total: 189 ms
Wall time: 233 ms


In [26]:
%%time

# fit the model
history = model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=20, batch_size=100)

Epoch 1/20
6/6 [==============================] - 7s 45ms/step - loss: 0.8820 - accuracy: 0.5830 - val_loss: 0.7673 - val_accuracy: 0.6351
Epoch 2/20
6/6 [==============================] - 0s 12ms/step - loss: 0.7543 - accuracy: 0.6293 - val_loss: 0.6951 - val_accuracy: 0.6441
Epoch 3/20
6/6 [==============================] - 0s 10ms/step - loss: 0.6745 - accuracy: 0.6467 - val_loss: 0.6293 - val_accuracy: 0.6577
Epoch 4/20
6/6 [==============================] - 0s 10ms/step - loss: 0.6180 - accuracy: 0.6390 - val_loss: 0.5900 - val_accuracy: 0.6622
Epoch 5/20
6/6 [==============================] - 0s 10ms/step - loss: 0.5885 - accuracy: 0.6834 - val_loss: 0.5848 - val_accuracy: 0.6667
Epoch 6/20
6/6 [==============================] - 0s 12ms/step - loss: 0.5666 - accuracy: 0.6969 - val_loss: 0.5735 - val_accuracy: 0.6532
Epoch 7/20
6/6 [==============================] - 0s 10ms/step - loss: 0.5762 - accuracy: 0.6351 - val_loss: 0.5574 - val_accuracy: 0.6757
Epoch 8/20
6/6 [===========

In [27]:
# evaluate the model

scores = model.evaluate(test_X, test_y, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.4467151165008545, 0.7702702879905701]

In [28]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.45
accuracy: 77.03%


In [29]:
y_pred = model.predict(test_X)
y_pred = (y_pred > 0.5)

c_matrix = confusion_matrix(test_y, y_pred)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Deep Neural Network", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)],  
                                                    'Precision': [TP/(TP+FP)],
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])


7/7 [==============================] - 0s 2ms/step


## Wide and Deep Neural Networks

In [30]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=20))
model.add(keras.layers.Dense(25, activation='relu',kernel_initializer= tf.keras.initializers.GlorotUniform()))
model.add(keras.layers.Dense(25, activation='relu',kernel_initializer= tf.keras.initializers.GlorotUniform()))
model.add(keras.layers.Dense(25, activation='relu',kernel_initializer= tf.keras.initializers.GlorotUniform()))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [31]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [32]:
# Fit the model

history = model.fit(train_X, train_y, 
                    validation_data=(test_X, test_y), 
                    epochs=20, batch_size=100)

Epoch 1/20
6/6 [==============================] - 3s 113ms/step - loss: 1.1805 - accuracy: 0.5116 - val_loss: 0.5327 - val_accuracy: 0.8018
Epoch 2/20
6/6 [==============================] - 0s 38ms/step - loss: 0.5099 - accuracy: 0.7355 - val_loss: 0.4692 - val_accuracy: 0.7658
Epoch 3/20
6/6 [==============================] - 0s 35ms/step - loss: 0.4156 - accuracy: 0.7857 - val_loss: 0.3740 - val_accuracy: 0.8333
Epoch 4/20
6/6 [==============================] - 0s 34ms/step - loss: 0.3574 - accuracy: 0.8301 - val_loss: 0.2990 - val_accuracy: 0.8874
Epoch 5/20
6/6 [==============================] - 0s 34ms/step - loss: 0.2942 - accuracy: 0.8726 - val_loss: 0.2679 - val_accuracy: 0.9189
Epoch 6/20
6/6 [==============================] - 0s 35ms/step - loss: 0.2317 - accuracy: 0.9131 - val_loss: 0.4967 - val_accuracy: 0.7117
Epoch 7/20
6/6 [==============================] - 0s 57ms/step - loss: 0.3065 - accuracy: 0.8436 - val_loss: 0.2079 - val_accuracy: 0.9459
Epoch 8/20
6/6 [==========

In [33]:
# evaluate the model

scores = model.evaluate(test_X, test_y, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.1499219387769699, 0.9234234094619751]

In [34]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.15
accuracy: 92.34%


In [35]:
y_pred = model.predict(test_X)
y_pred = (y_pred > 0.5)

c_matrix = confusion_matrix(test_y, y_pred)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Wide and Deep Neural Network", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)],  
                                                    'Precision': [TP/(TP+FP)],
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

7/7 [==============================] - 0s 2ms/step


## RandomGrid Search

In [36]:
%%time

!pip install scikeras
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=20)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotUniform(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    ann.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    return ann


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CPU times: user 43.4 ms, sys: 40.2 ms, total: 83.6 ms
Wall time: 3.59 s


In [37]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=20,
    dropout = 0.0
)


In [38]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':["adam",'sgd']
}
keras_clf.get_params().keys()



dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [39]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(train_X, train_y, callbacks=callback, verbose=0)


2/2 [==============================] - 0s 8ms/step


2/2 [==============================] - 0s 5ms/step


6/6 [==============================] - 0s 3ms/step


In [40]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.005,
 'optimizer': 'adam',
 'model__hidden_layer_sizes': (100,),
 'model__dropout': 0,
 'epochs': 100,
 'batch_size': 20}

In [41]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.005, 'optimizer': 'adam', 'model__hidden_layer_sizes': (100,), 'model__dropout': 0, 'epochs': 100, 'batch_size': 20}


In [42]:
%%time
y_pred = best_net.predict(test_X)
print(classification_report(test_y, y_pred))

12/12 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.93      0.99      0.96        86
           1       0.99      0.96      0.97       136

    accuracy                           0.97       222
   macro avg       0.96      0.97      0.97       222
weighted avg       0.97      0.97      0.97       222

CPU times: user 123 ms, sys: 3.21 ms, total: 126 ms
Wall time: 147 ms


In [43]:
y_pred = model.predict(test_X)
y_pred = (y_pred > 0.5)

c_matrix = confusion_matrix(test_y, y_pred)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Deep Neural Network RandomGrid Search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)],  
                                                    'Precision': [TP/(TP+FP)],
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

7/7 [==============================] - 0s 3ms/step


## PERFORMANCE OF ALL MODELS

In [44]:
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.986486,1.000000,0.977941,0.988848
0,SVM,1.000000,1.000000,1.000000,1.000000
0,Decision Tree,1.000000,1.000000,1.000000,1.000000
0,Neural_Networks_Grid_search,0.995495,1.000000,0.992647,0.996310
0,Deep Neural Network,0.770270,0.754491,0.926471,0.831683
0,Wide and Deep Neural Network,0.923423,0.899329,0.985294,0.940351
0,Deep Neural Network RandomGrid Search,0.923423,0.899329,0.985294,0.940351


In [45]:
performance.sort_values(by=['Accuracy'])

,model,Accuracy,Precision,Recall,F1
0,Deep Neural Network,0.770270,0.754491,0.926471,0.831683
0,Wide and Deep Neural Network,0.923423,0.899329,0.985294,0.940351
0,Deep Neural Network RandomGrid Search,0.923423,0.899329,0.985294,0.940351
0,Logistic Regression,0.986486,1.000000,0.977941,0.988848
0,Neural_Networks_Grid_search,0.995495,1.000000,0.992647,0.996310
0,SVM,1.000000,1.000000,1.000000,1.000000
0,Decision Tree,1.000000,1.000000,1.000000,1.000000


## ANALYSIS OF MODELS 

##Summary and Analysis: 

The problem statement is related to the Human Resources(HR) Analytics where the employee's absence can be validated using various reasons which can potentially impact the weightage for the decision of promotion and  elimination at the company.

Here, the False Negative (where we predict the employee is not absent but in actuality, he is absent) and False Positive (where we predict the employee is absent but in actuality, he is not absent) both of them have the equal priority as it may help in adding weightage at the decision making process for the promotion or elimination in the company.
Hence, The **Accuracy** is considered here as the main parameter for the decision of the best fitting model for the data chosen.


**Impact from Preprocessing data:**
The 3 predictors are label encoded leading to many zeroes and ones in the data while processing. The resulted data from pre processing is encoded, splitted, balanced, regularized and early stopped for better training and reduce overfitting while fitting the data set into these models. The regularization and early stopping is introduced for the reduction of over fitting as mentioned in the python cheatsheet from the ppt given in the modules.The data contains 741 rows(limited dataset) and is very well trained and contains more zeros and ones because of the encoding and scaling. Hence the scoring parameters accuracy is high for the model fitted. If the dataset had many entries, then the training and testing can be effective compatred to a limited entry dataset.



The **performance** of distinct machine learning models—Logistic Regression, Support Vector Machine (SVM), Decision Tree, Neural network, Deep Neural network and Wide&Deep Neural Network (MLP and keras) are evaluated.

The Deep Neural Network, Wide and Deep Neural Network, and Deep Neural Network RandomGrid Search have considerably lower accuracy and F1 scores in comparison to the other models, as can be seen from the evaluation metrics. They are able to accurately recognize more positive examples, or true positives. This is because these models are more complicated and are better able to identify complex patterns in the data. The Logistic Regression, SVM, Decision Tree, and both Neural Networks Grid search models, on the other hand, have flawless accuracy, precision, recall, and F1 scores, demonstrating that they were able to properly categorize all cases in the dataset. For the svm, decision tree and Neural networks(MLP), the model is overfitting after hypertuning also. Wide & deep neural network has accuracy to 92% where as the **Neural networks with grid search (MLP)** is performing good with 99% accuracy and appears to be the best model for this specific task. It's crucial to remember that these metrics only give a partial picture of the model's performance, and it's likely that model interpretability or computing efficiency may also be crucial elements to take into account when choosing a model but for our business model, the Neural networks (MLP) suits well and performing better without overfitting.